In [95]:
%config IPCompleter.greedy=True
from __future__ import print_function
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools
from receipt_detector import is_receipt
from datetime import datetime,timedelta
from dateutil.parser import parse
import pytz


In [83]:
# If modifying these scopes, delete the file token.json.
SCOPES = 'https://www.googleapis.com/auth/gmail.readonly'

In [84]:
store = file.Storage('token.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('credentials.json', SCOPES)
    creds = tools.run_flow(flow, store)
service = build('gmail', 'v1', http=creds.authorize(Http()))


In [85]:
user_id =  'me'
label_id_one = 'INBOX'
label_id_two = 'UNREAD'

# Getting all the unread messages from Inbox
# labelIds can be changed accordingly
unread_msgs = service.users().messages().list(userId='me',labelIds=[label_id_one]).execute()

# We get a dictonary. Now reading values for the key 'messages'
mssg_list = unread_msgs['messages']

print ("Total unread messages in inbox: ", str(len(mssg_list)))

final_list = [ ]

Total unread messages in inbox:  100


In [86]:
MAX_MESSAGES = 100

In [87]:

    
try:
    response = service.users().messages().list(userId=user_id,labelIds=[label_id_one]).execute()
    messages = []
    if 'messages' in response:
      messages.extend(response['messages'])

    while 'nextPageToken' in response and len(messages) <= MAX_MESSAGES:
      page_token = response['nextPageToken']
      response = service.users().messages().list(userId=user_id,labelIds=[label_id_one],
                                         pageToken=page_token).execute()
      messages.extend(response['messages'])
      print(len(messages))

except error:
    print ('An error occurred: %s' % error)

200


In [107]:
utc=pytz.UTC


def grab_all_receipts(days):
    
    date_N_days_ago = datetime.now().replace(tzinfo=utc) - timedelta(days=days)
    
    for mssg in messages:
        temp_dict = { }
        m_id = mssg['id'] # get id of individual message
        message = service.users().messages().get(userId=user_id, id=m_id).execute() # fetch the message using API
        payld = message['payload'] # get payload of the message 
        headr = payld['headers'] # get header of the payload

        for one in headr: # getting the Subject
            if one['name'] == 'Subject':
                msg_subject = one['value']
                if is_receipt(msg_subject):
                    print(msg_subject)
            if one['name'] == 'Date':
                date_string = one['value']
                d = parse(date_string)
                if (d < date_N_days_ago):
                    return
            
grab_all_receipts(2)

2018-09-26 12:40:27.765757+00:00 / 2018-09-28 11:52:48-07:00
2018-09-26 12:40:27.765757+00:00 / 2018-09-28 18:31:49+00:00
2018-09-26 12:40:27.765757+00:00 / 2018-09-28 17:27:17+00:00
2018-09-26 12:40:27.765757+00:00 / 2018-09-28 17:14:21+00:00
2018-09-26 12:40:27.765757+00:00 / 2018-09-28 09:29:43-07:00
China 8-Night Trip: $398 with Seattle Flights
2018-09-26 12:40:27.765757+00:00 / 2018-09-28 14:55:03+00:00
2018-09-26 12:40:27.765757+00:00 / 2018-09-28 14:51:52+00:00
2018-09-26 12:40:27.765757+00:00 / 2018-09-28 14:35:37+00:00
2018-09-26 12:40:27.765757+00:00 / 2018-09-28 11:44:04+00:00
LAST DAY! Use this code to get 10% off your next booking
2018-09-26 12:40:27.765757+00:00 / 2018-09-28 05:46:56-06:00
2018-09-26 12:40:27.765757+00:00 / 2018-09-28 00:01:14-05:00
2018-09-26 12:40:27.765757+00:00 / 2018-09-28 03:23:35+00:00
2018-09-26 12:40:27.765757+00:00 / 2018-09-28 03:19:02+00:00
2018-09-26 12:40:27.765757+00:00 / 2018-09-28 03:09:34+00:00
2018-09-26 12:40:27.765757+00:00 / 2018-09-